In [12]:
import json
import os

NUM_DATA = 100
NUM_RESPONSES = 10
BATCH_SIZE = 16

NUM_EPOCHS = 10

NUM_TOP = 2

PREF_MODEL = "vectorzhou/gemma-2-2b-it-preference_dataset_mixture2_and_safe_pku-Preference"

# BASE_NAMES = {
#     "vectorzhou/gemma-2-2b-it-alpaca-cleaned-SFT-PKU-SafeRLHF-OnlineIPO1-lora-0227213453": "\\oipoone",
#     "vectorzhou/gemma-2-2b-it-alpaca-cleaned-SFT-PKU-SafeRLHF-OnlineIPO2-lora-0227214805": "\\oipotwo",
#     "vectorzhou/gemma-2-2b-it-alpaca-cleaned-SFT-PKU-SafeRLHF-NashMD-lora-0227215018": "\\nmd",
#     "vectorzhou/gemma-2-2b-it-alpaca-cleaned-SFT-PKU-SafeRLHF-NashMDPG-lora-0301154042": "\\nmdpg",
#     "dummy/new_MPO": "\\mpo",
#     "vectorzhou/gemma-2-2b-it-alpaca-cleaned-SFT-PKU-SafeRLHF-Extragradient-lora-0224142549": "\\eg",
# }
# REF_MODEL = "vectorzhou/gemma-2-2b-it-alpaca-cleaned-SFT"

BASE_NAMES = {
    "vectorzhou/vectorzhou-Qwen2-5-1-5B-Instruct-SFT-OpenHerm-ction-v0-1-OnlineIPO1-lora-0603201242": "OIPO1",
    "vectorzhou/vectorzhou-Qwen2-5-1-5B-Instruct-SFT-OpenHerm-ction-v0-1-OnlineIPO2-lora-0604063354": "OIPO2",
    "vectorzhou/vectorzhou-Qwen2-5-1-5B-Instruct-SFT-OpenHerm-ollection-v0-1-NashMD-lora-0603201151": "NMD",
    "vectorzhou/Qwen2.5-1.5B-Instruct-SFT-OpenHerm-llection-v0.1-NashMDPG-lora-0605154608": "NMDPG",
    "vectorzhou/vectorzhou-Qwen2-5-1-5B-Instruct-SFT-OpenHerm-on-v0-1-Extragradient-lora-0604122854": "EGPO",
}
REF_MODEL = "vectorzhou/Qwen2.5-1.5B-Instruct-SFT-OpenHermes-2.5-Standard-SFT"

ALL_MODEL_DICT = BASE_NAMES.copy()
ALL_MODEL_DICT.update({REF_MODEL: "$\\pi_\\tref$"})

def get_pref(compare_results, model1, model2, judge_type):
    model1 = model1.split("/")[-1]
    model2 = model2.split("/")[-1]

    try:
        pair_results = compare_results[model1][model2]
        flip = False
    except:
        pair_results = compare_results[model2][model1]
        flip = True

    avg = sum([sum([x[judge_type] for x in pair_results[k]["results"]]) for k in range(NUM_DATA)]) / (NUM_DATA * NUM_RESPONSES)
    if flip:
        avg = 1 - avg
    
    return avg

In [25]:
fn = os.path.join("eval_results", "compare.json")
with open(fn) as f:
    compare_results = json.load(f)

all_models = {}

for base_name in BASE_NAMES:
    win_rates = {}
    for epoch in range(NUM_EPOCHS):
        model1 = f"{base_name}-epoch-{epoch+1}"
        win_rates[epoch] = get_pref(compare_results, model1, REF_MODEL, "pref_model")
    sorted_win_rates = sorted(win_rates.items(), key=lambda x: x[1], reverse=True)
    picked_ckpts = [sorted_win_rates[k][0] for k in range(NUM_TOP)]
    picked_ckpts.sort()

    all_models[base_name] = picked_ckpts

for k, v in all_models.items():
    print(k, v)


vectorzhou/vectorzhou-Qwen2-5-1-5B-Instruct-SFT-OpenHerm-ction-v0-1-OnlineIPO1-lora-0603201242 [6, 8]
vectorzhou/vectorzhou-Qwen2-5-1-5B-Instruct-SFT-OpenHerm-ction-v0-1-OnlineIPO2-lora-0604063354 [4, 7]
vectorzhou/vectorzhou-Qwen2-5-1-5B-Instruct-SFT-OpenHerm-ollection-v0-1-NashMD-lora-0603201151 [5, 6]
vectorzhou/Qwen2.5-1.5B-Instruct-SFT-OpenHerm-llection-v0.1-NashMDPG-lora-0605154608 [1, 3]
vectorzhou/vectorzhou-Qwen2-5-1-5B-Instruct-SFT-OpenHerm-on-v0-1-Extragradient-lora-0604122854 [7, 9]


In [18]:
m1 = list(compare_results.keys())
print("vectorzhou-Qwen2-5-1-5B-Instruct-SFT-OpenHerm-ction-v0-1-OnlineIPO1-lora-0603201242-epoch-7" in m1)

print("vectorzhou-Qwen2-5-1-5B-Instruct-SFT-OpenHerm-ction-v0-1-OnlineIPO1-lora-0603201242-epoch-7" in compare_results["vectorzhou-Qwen2-5-1-5B-Instruct-SFT-OpenHerm-ction-v0-1-OnlineIPO2-lora-0604063354-epoch-5"])

print()

True
False



In [ ]:
# Latex

model2_list = [REF_MODEL] + list(all_models.keys())

# write first line
print("\\begin{tabular}{" + "cc||c" + ("|" + "c" * NUM_TOP) * (len(model2_list) - 1) + "}")
print("\\toprule")
print("ALG &  & \multirow{2}{*}{$\\pi_\\tref$} ", end="")
for i in range(1, len(model2_list)):
    model = model2_list[i]
    sep = "|" if i != len(model2_list) - 1 else ""
    print(f"& \multicolumn{{{NUM_TOP}}}{{c{sep}}}{{{ALL_MODEL_DICT[model]}}} ", end="")
print("\\\\")
print(" & Ep & ", end="")
for model in model2_list[1:]:
    for epoch in all_models[model]:
        print(f"& {epoch + 1} ", end="")
print("\\\\")
print("\\hhline{" + "=" * (2 + 1 + NUM_TOP * (len(model2_list) - 1)) + "}")

for i, model1 in enumerate(all_models.keys()):
    if i > 0:
        print("\\hline")
    print(f"\multirow{{{NUM_TOP}}}{{*}}{{{BASE_NAMES[model1]}}}", end="")
    for epoch in all_models[model1]:
        print(f" & {epoch + 1}", end="")
        for model2 in model2_list:
            if model2 == REF_MODEL:
                epochs = [None]
            else:
                epochs = all_models[model2]

            for epoch2 in epochs:
                if model1 == model2:
                    print(" &", end="")
                    continue

                model2_name = f"{model2}-epoch-{epoch2+1}" if epoch2 is not None else model2

                avg = get_pref(compare_results, f"{model1}-epoch-{epoch+1}", model2_name, "pref_model")

                if avg > 0.5:
                    bfl, bfr = "\\red{\\boldsymbol{", "}}"
                else:
                    bfl, bfr = "", ""
                
                print(" & $" + f"{bfl}{avg:.1%}{bfr}$".replace("%", "\\%"), end="")
        print("\\\\")

print("\\bottomrule")
print("\\end{tabular}")

\begin{tabular}{cc||c|cc|cc|cc|cc|cc}
\toprule
ALG &  & \multirow{2}{*}{$\pi_\tref$} & \multicolumn{2}{c|}{OIPO1} & \multicolumn{2}{c|}{OIPO2} & \multicolumn{2}{c|}{NMD} & \multicolumn{2}{c|}{NMDPG} & \multicolumn{2}{c}{EGPO} \\
 & Ep & & 7 & 9 & 5 & 8 & 6 & 7 & 2 & 4 & 8 & 10 \\
\hhline{=============}
\multirow{2}{*}{OIPO1} & 7 & $\red{\boldsymbol{50.5\%}}$ & & & $49.8\%$ & $49.8\%$ & $49.8\%$ & $49.8\%$ & $\red{\boldsymbol{51.3\%}}$ & $\red{\boldsymbol{51.1\%}}$ & $45.5\%$ & $44.8\%$\\
 & 9 & $\red{\boldsymbol{50.1\%}}$ & & & $49.1\%$ & $49.8\%$ & $49.4\%$ & $49.4\%$ & $\red{\boldsymbol{52.4\%}}$ & $\red{\boldsymbol{52.2\%}}$ & $46.8\%$ & $47.0\%$\\
\hline
\multirow{2}{*}{OIPO2} & 5 & $49.1\%$ & $\red{\boldsymbol{50.2\%}}$ & $\red{\boldsymbol{50.9\%}}$ & & & $49.5\%$ & $49.6\%$ & $\red{\boldsymbol{51.9\%}}$ & $\red{\boldsymbol{51.7\%}}$ & $46.0\%$ & $45.5\%$\\
 & 8 & $\red{\boldsymbol{50.3\%}}$ & $\red{\boldsymbol{50.2\%}}$ & $\red{\boldsymbol{50.2\%}}$ & & & $49.2\%$ & $49.5\%$ & $\

In [33]:
# --- helpers ---------------------------------------------------------------
def fmt(avg: float) -> str:
    """Format a preference value as X.Y% (bold if >50%)."""
    cell = f"{avg:.1%}"
    return f"**{cell}**" if avg > 0.5 else cell


# --- header rows -----------------------------------------------------------
model2_list = [REF_MODEL] + list(all_models.keys())

# Row 1: ALG / π_ref / model names
header1 = ["ALG", "Ep", "$\pi_\\text{ref}$"]
for m in model2_list[1:]:
    epochs = all_models[m]
    for j in range(len(epochs)):
        header1.append(ALL_MODEL_DICT[m] if j == 0 else "")

# Row 2: blank / "Ep" / epoch numbers
header2 = ["", "", ""]
for m in model2_list[1:]:
    header2.extend([str(e + 1) for e in all_models[m]])

# Markdown divider (--- under every column)
divider = ["---"] * len(header1)

# Print header
print("| " + " | ".join(header1) + " |")
print("| " + " | ".join(divider) + " |")
print("| " + " | ".join(header2) + " |")

# --- body rows -------------------------------------------------------------
for i, (model1, epochs1) in enumerate(all_models.items()):
    for r, epoch1 in enumerate(epochs1):
        row = []

        # Column 0-1: algorithm name (only on first row of the block) & epoch index
        row.append(BASE_NAMES[model1] if r == 0 else "")
        row.append(str(epoch1 + 1))

        # Column 2: π_ref preference
        pref_ref = get_pref(compare_results,
                            f"{model1}-epoch-{epoch1}",
                            REF_MODEL,
                            "pref_model")
        row.append(fmt(pref_ref))

        # Columns 3+: comparisons with every other model / epoch
        for model2 in model2_list[1:]:
            for epoch2 in all_models[model2]:
                if model1 == model2:          # diagonal
                    row.append("")
                    continue
                pref = get_pref(compare_results,
                                f"{model1}-epoch-{epoch1}",
                                f"{model2}-epoch-{epoch2}",
                                "pref_model")
                row.append(fmt(pref))

        print("| " + " | ".join(row) + " |")


| ALG | Ep | $\pi_\text{ref}$ | OIPO1 |  | OIPO2 |  | NMD |  | NMDPG |  | EGPO |  |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
|  |  |  | 7 | 9 | 5 | 8 | 6 | 7 | 2 | 4 | 8 | 10 |
| OIPO1 | 7 | **50.5%** |  |  | 49.8% | 49.8% | 49.8% | 49.8% | **51.3%** | **51.1%** | 45.5% | 44.8% |
|  | 9 | **50.1%** |  |  | 49.1% | 49.8% | 49.4% | 49.4% | **52.4%** | **52.2%** | 46.8% | 47.0% |
| OIPO2 | 5 | 49.1% | **50.2%** | **50.9%** |  |  | 49.5% | 49.6% | **51.9%** | **51.7%** | 46.0% | 45.5% |
|  | 8 | **50.3%** | **50.2%** | **50.2%** |  |  | 49.2% | 49.5% | **52.5%** | **51.1%** | 45.7% | 45.0% |
| NMD | 6 | **50.7%** | **50.2%** | **50.6%** | **50.5%** | **50.8%** |  |  | **52.5%** | **51.1%** | 44.7% | 46.2% |
|  | 7 | **51.7%** | **50.2%** | **50.6%** | **50.4%** | **50.5%** |  |  | **52.0%** | **51.8%** | 47.0% | 46.8% |
| NMDPG | 2 | 49.1% | 48.7% | 47.6% | 48.1% | 47.5% | 47.5% | 48.0% |  |  | 44.0% | 43.1% |
|  | 4 | 49.5% | 48.9% | 47.8% | 48.3% | 

In [ ]:
# Latex
# write first line
print("\\begin{tabular}{" + "cc|c||" * (len(BASE_NAMES) - 1) + "cc|c}")
print("\\toprule")
for i in range(len(BASE_NAMES)):
    print(("& " if i > 0 else "") + "ALG & Ep & $\\pi_\\tref$ ", end="")
print("\\\\")
print("\hhline{" + "=" * (3 * len(BASE_NAMES)) + "}")


for epoch in range(NUM_EPOCHS):
    for i, model1 in enumerate(BASE_NAMES):
        if i > 0:
            print(" & ", end="")
        if epoch == 0:
            print(f"\multirow{{{NUM_EPOCHS}}}{{*}}{{{BASE_NAMES[model1]}}}", end="")
        print(f" & {epoch + 1}", end="")

        avg = get_pref(compare_results, f"{model1}-epoch-{epoch + 1}", REF_MODEL, "pref_model") # TODO

        if epoch in all_models[model1]:
            bfl, bfr = "\\red{\\boldsymbol{", "}}"
        else:
            bfl, bfr = "", ""
        
        print(" & $" + f"{bfl}{avg:.1%}{bfr}$".replace("%", "\\%"), end="")
    print("\\\\")

print("\\bottomrule")
print("\\end{tabular}")

\begin{tabular}{cc|c||cc|c||cc|c||cc|c||cc|c}
\toprule
ALG & Ep & $\pi_\tref$ & ALG & Ep & $\pi_\tref$ & ALG & Ep & $\pi_\tref$ & ALG & Ep & $\pi_\tref$ & ALG & Ep & $\pi_\tref$ \\
\hhline{===============}
\multirow{10}{*}{OIPO1} & 1 & $50.2\%$ & \multirow{10}{*}{OIPO2} & 1 & $47.9\%$ & \multirow{10}{*}{NMD} & 1 & $49.7\%$ & \multirow{10}{*}{NMDPG} & 1 & $49.1\%$ & \multirow{10}{*}{EGPO} & 1 & $49.8\%$\\
 & 2 & $50.8\%$ &  & 2 & $49.7\%$ &  & 2 & $51.6\%$ &  & 2 & $\red{\boldsymbol{50.1\%}}$ &  & 2 & $51.2\%$\\
 & 3 & $49.8\%$ &  & 3 & $50.4\%$ &  & 3 & $51.6\%$ &  & 3 & $49.5\%$ &  & 3 & $50.0\%$\\
 & 4 & $48.3\%$ &  & 4 & $49.1\%$ &  & 4 & $51.4\%$ &  & 4 & $\red{\boldsymbol{51.9\%}}$ &  & 4 & $52.4\%$\\
 & 5 & $50.2\%$ &  & 5 & $\red{\boldsymbol{51.4\%}}$ &  & 5 & $50.7\%$ &  & 5 & $50.1\%$ &  & 5 & $54.4\%$\\
 & 6 & $50.5\%$ &  & 6 & $50.7\%$ &  & 6 & $\red{\boldsymbol{51.7\%}}$ &  & 6 & $49.6\%$ &  & 6 & $54.7\%$\\
 & 7 & $\red{\boldsymbol{51.2\%}}$ &  & 7 & $50.3\%$ &  & 7 & $\re

In [34]:
# -------------------------------------------------------------------------
# Helpers
# -------------------------------------------------------------------------
def fmt(avg: float, highlight: bool) -> str:
    """Format 0.0–1.0 as 'xx.x%' and bold it if highlight == True."""
    txt = f"{avg:.1%}"
    return f"**{txt}**" if highlight else txt


# -------------------------------------------------------------------------
# Header construction
# -------------------------------------------------------------------------
# Three columns per model block: ALG | Ep | π_ref
header = []
for _ in BASE_NAMES:
    header += ["ALG", "Ep", "$\pi_\\text{ref}$"]

divider = ["---"] * len(header)

print("| " + " | ".join(header) + " |")
print("| " + " | ".join(divider) + " |")

# -------------------------------------------------------------------------
# Body rows
# -------------------------------------------------------------------------
for epoch in range(NUM_EPOCHS):
    row_cells = []

    for i, model in enumerate(BASE_NAMES):
        # Column 1: ALG (only on the first epoch row for each model block)
        row_cells.append(ALL_MODEL_DICT[model] if epoch == 0 else "")
        # Column 2: Ep
        row_cells.append(str(epoch + 1))
        # Column 3: π_ref (bold if this epoch exists in all_models[model])
        avg = get_pref(
            compare_results,
            f"{model}-epoch-{epoch + 1}",
            REF_MODEL,
            "pref_model"
        )
        highlight = epoch in all_models[model]
        row_cells.append(fmt(avg, highlight))

    print("| " + " | ".join(row_cells) + " |")


| ALG | Ep | $\pi_\text{ref}$ | ALG | Ep | $\pi_\text{ref}$ | ALG | Ep | $\pi_\text{ref}$ | ALG | Ep | $\pi_\text{ref}$ | ALG | Ep | $\pi_\text{ref}$ |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| OIPO1 | 1 | 50.2% | OIPO2 | 1 | 47.9% | NMD | 1 | 49.7% | NMDPG | 1 | 49.1% | EGPO | 1 | 49.8% |
|  | 2 | 50.8% |  | 2 | 49.7% |  | 2 | 51.6% |  | 2 | **50.1%** |  | 2 | 51.2% |
|  | 3 | 49.8% |  | 3 | 50.4% |  | 3 | 51.6% |  | 3 | 49.5% |  | 3 | 50.0% |
|  | 4 | 48.3% |  | 4 | 49.1% |  | 4 | 51.4% |  | 4 | **51.9%** |  | 4 | 52.4% |
|  | 5 | 50.2% |  | 5 | **51.4%** |  | 5 | 50.7% |  | 5 | 50.1% |  | 5 | 54.4% |
|  | 6 | 50.5% |  | 6 | 50.7% |  | 6 | **51.7%** |  | 6 | 49.6% |  | 6 | 54.7% |
|  | 7 | **51.2%** |  | 7 | 50.3% |  | 7 | **53.2%** |  | 7 | 50.0% |  | 7 | 54.2% |
|  | 8 | 50.1% |  | 8 | **52.5%** |  | 8 | 51.3% |  | 8 | 50.1% |  | 8 | **55.1%** |
|  | 9 | **50.9%** |  | 9 | 50.8% |  | 9 | 51.0% |  | 9 | 49.7% |  | 9 | 54.2% |
|  | 